In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Pruning").getOrCreate()

In [ ]:
## Sample Data

data = [
    ("Alice", "HR", 1000),
    ("Bob", "IT", 1500),
    ("Charlie", "HR", 1200),
    ("David", "IT", 1800),
    ("Eve", "Finance", 2000),
    ("Frank", "Finance", 1800)
]

columns = ["employee_name", "department", "salary"]

In [ ]:
## Creating Data Frame
df = spark.createDataFrame(data, columns)

In [ ]:
### Write DataFrame using partitioning
output_path = "/content/sample_data/OutputData"

df.write\
  .mode("overwrite")\
  .partitionBy("department")\
  .parquet(output_path)

In [ ]:
### Write DataFrame using without partitioning
output_path_new = "/content/sample_data/OutputDataWithout"

df.write\
  .mode("overwrite")\
  .parquet(output_path_new)

In [ ]:
df_withpart = spark.read.format("parquet").load(output_path)

In [ ]:
df_withpart.show()

+-------------+------+----------+
|employee_name|salary|department|
+-------------+------+----------+
|        Alice|  1000|        HR|
|      Charlie|  1200|        HR|
|        David|  1800|        IT|
|          Eve|  2000|   Finance|
|        Frank|  1800|   Finance|
|          Bob|  1500|        IT|
+-------------+------+----------+



In [ ]:
df_withpart.explain(True)

== Parsed Logical Plan ==
Relation [employee_name#132,salary#133L,department#134] parquet

== Analyzed Logical Plan ==
employee_name: string, salary: bigint, department: string
Relation [employee_name#132,salary#133L,department#134] parquet

== Optimized Logical Plan ==
Relation [employee_name#132,salary#133L,department#134] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [employee_name#132,salary#133L,department#134] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/sample_data/OutputData], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<employee_name:string,salary:bigint>



In [ ]:
from pyspark.sql.functions import *

In [ ]:
df_withpart = spark.read.format("parquet").load(output_path)

In [ ]:
df_withpart.show()

+-------------+------+----------+
|employee_name|salary|department|
+-------------+------+----------+
|        Alice|  1000|        HR|
|      Charlie|  1200|        HR|
+-------------+------+----------+



In [ ]:
df_withpart.explain(True)

== Parsed Logical Plan ==
'Filter ('department = HR)
+- Relation [employee_name#152,salary#153L,department#154] parquet

== Analyzed Logical Plan ==
employee_name: string, salary: bigint, department: string
Filter (department#154 = HR)
+- Relation [employee_name#152,salary#153L,department#154] parquet

== Optimized Logical Plan ==
Filter (isnotnull(department#154) AND (department#154 = HR))
+- Relation [employee_name#152,salary#153L,department#154] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [employee_name#152,salary#153L,department#154] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/sample_data/OutputData], PartitionFilters: [isnotnull(department#154), (department#154 = HR)], PushedFilters: [], ReadSchema: struct<employee_name:string,salary:bigint>



In [ ]:
df_withoutpart = spark.read.format("parquet").load(output_path_new).filter(col("department") == "HR")

In [ ]:
df_withoutpart.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Alice|        HR|  1000|
|      Charlie|        HR|  1200|
+-------------+----------+------+



In [ ]:
df_withpart.show()

+-------------+------+----------+
|employee_name|salary|department|
+-------------+------+----------+
|        Alice|  1000|        HR|
|      Charlie|  1200|        HR|
|        David|  1800|        IT|
|          Eve|  2000|   Finance|
|        Frank|  1800|   Finance|
|          Bob|  1500|        IT|
+-------------+------+----------+



# **Dynamic Partition Pruning**

In [ ]:
df_join = df_withoutpart.join(df_withpart,
                              (df_withoutpart['employee_name'] == df_withpart['employee_name']) &
                               (df_withoutpart['department']== df_withpart['department']), "inner")

In [ ]:
df_join.show()

+-------------+----------+------+-------------+------+----------+
|employee_name|department|salary|employee_name|salary|department|
+-------------+----------+------+-------------+------+----------+
|        Alice|        HR|  1000|        Alice|  1000|        HR|
|      Charlie|        HR|  1200|      Charlie|  1200|        HR|
+-------------+----------+------+-------------+------+----------+



In [ ]:
df_join.explain(True)

== Parsed Logical Plan ==
Join Inner, ((employee_name#175 = employee_name#195) AND (department#176 = department#197))
:- Filter (department#176 = HR)
:  +- Relation [employee_name#175,department#176,salary#177L] parquet
+- Relation [employee_name#195,salary#196L,department#197] parquet

== Analyzed Logical Plan ==
employee_name: string, department: string, salary: bigint, employee_name: string, salary: bigint, department: string
Join Inner, ((employee_name#175 = employee_name#195) AND (department#176 = department#197))
:- Filter (department#176 = HR)
:  +- Relation [employee_name#175,department#176,salary#177L] parquet
+- Relation [employee_name#195,salary#196L,department#197] parquet

== Optimized Logical Plan ==
Join Inner, ((employee_name#175 = employee_name#195) AND (department#176 = department#197))
:- Filter ((isnotnull(department#176) AND (department#176 = HR)) AND isnotnull(employee_name#175))
:  +- Relation [employee_name#175,department#176,salary#177L] parquet
+- Filter (((de